# Satellite Interference Analysis for RFI Pipeline Data

This notebook analyzes RFI pipeline output files and correlates them with Starlink satellite positions to identify potential interference sources. We'll focus on observations in the 10.6-11.2 GHz range and check for satellites within 1 degree of the observation beam.

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
from pathlib import Path
import random
import warnings
import requests
from io import StringIO
warnings.filterwarnings('ignore')

# Skyfield for satellite tracking
from skyfield.api import load, Topos, utc
from skyfield.timelib import Time

# For angular separation calculations
from skyfield.units import Angle

print("Libraries imported successfully!")
print(f"Working directory: {Path.cwd()}")

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

Libraries imported successfully!
Working directory: /mnt_home/andresl/rfi-pipeline/experiments


## 2. Load and Parse RFI Pipeline Data

In [2]:
# Path to the files.csv from RFI pipeline run
files_csv_path = "/datax/scratch/andresl/pipeline-runs/all-bands/full-run-60k/files.csv"

print(f"Loading data from: {files_csv_path}")

# Load the files.csv
df = pd.read_csv(files_csv_path)

print(f"Loaded {len(df)} files")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())

# Parse timestamps and convert to datetime objects
df['time_completed_dt'] = pd.to_datetime(df['time_completed'])

# Calculate frequency range for each file (in MHz)
# fch1 is the frequency of the first channel, foff is the frequency step
# flch is the frequency of the last channel (fch1 + foff * nchans)
df['freq_start_mhz'] = df['flch']  # Lower frequency bound
df['freq_end_mhz'] = df['fch1']    # Upper frequency bound

print(f"\nFrequency range summary:")
print(f"Min frequency: {df['freq_start_mhz'].min():.2f} MHz")
print(f"Max frequency: {df['freq_end_mhz'].max():.2f} MHz")

Loading data from: /datax/scratch/andresl/pipeline-runs/all-bands/full-run-60k/files.csv
Loaded 60795 files
Columns: ['file', 'num_hits', 'time_completed', 'RA', 'DEC', 'tstart', 'nchans', 'foff', 'fch1', 'flch']

First few rows:
                                                file  num_hits  \
0  /datag/pipeline/AGBT24B_999_21/blc04_blp04/blc...         0   
1  /datag/pipeline/AGBT22B_999_41/blc02_blp02/blc...         0   
2  /datag/pipeline/AGBT24A_999_07/blc06_blp06/blc...        87   
3  /datag/pipeline/AGBT24A_999_20/blc20_blp10/blc...        34   
4  /datag/pipeline/AGBT23B_999_38/blc61_blp21/blc...         8   

                     time_completed         RA      DEC        tstart  \
0  2025-08-03T06:49:30.027120+00:00   4.035467 -10.9115  60652.104861   
1  2025-08-03T06:49:30.755366+00:00  12.424133  13.1221  60054.954826   
2  2025-08-03T06:49:44.751474+00:00  19.220073  76.1255  60384.226481   
3  2025-08-03T06:49:48.961125+00:00  22.853027  48.7344  60415.838333   
4  2025-

## 3. Filter Files by Frequency Range

We want to find files with significant overlap with the 10.6-11.2 GHz range.

In [3]:
# Target frequency range (in MHz)
target_freq_min = 10600  # 10.6 GHz
target_freq_max = 11200  # 11.2 GHz

print(f"Target frequency range: {target_freq_min}-{target_freq_max} MHz")

# Function to calculate overlap between two frequency ranges
def calculate_overlap(start1, end1, start2, end2):
    """Calculate the overlap between two frequency ranges"""
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    overlap = max(0, overlap_end - overlap_start)
    return overlap

# Calculate overlap for each file
df['overlap_mhz'] = df.apply(
    lambda row: calculate_overlap(
        row['freq_start_mhz'], row['freq_end_mhz'],
        target_freq_min, target_freq_max
    ), axis=1
)

# Calculate overlap percentage (relative to target range)
target_range_width = target_freq_max - target_freq_min
df['overlap_percentage'] = (df['overlap_mhz'] / target_range_width) * 100

# Filter files with significant overlap (e.g., >50% overlap)
min_overlap_percentage = 50
filtered_df = df[df['overlap_percentage'] >= min_overlap_percentage].copy()

print(f"\nFound {len(filtered_df)} files with ≥{min_overlap_percentage}% overlap")
print(f"Overlap percentage range: {filtered_df['overlap_percentage'].min():.1f}% - {filtered_df['overlap_percentage'].max():.1f}%")

if len(filtered_df) > 0:
    print(f"\nTop files by overlap percentage:")
    top_overlap = filtered_df.nlargest(5, 'overlap_percentage')[['file', 'freq_start_mhz', 'freq_end_mhz', 'overlap_percentage']]
    for idx, row in top_overlap.iterrows():
        filename = Path(row['file']).name
        print(f"  {filename}: {row['freq_start_mhz']:.1f}-{row['freq_end_mhz']:.1f} MHz ({row['overlap_percentage']:.1f}% overlap)")
else:
    print("No files found with sufficient overlap. Reducing threshold...")
    min_overlap_percentage = 10
    filtered_df = df[df['overlap_percentage'] >= min_overlap_percentage].copy()
    print(f"Found {len(filtered_df)} files with ≥{min_overlap_percentage}% overlap")

Target frequency range: 10600-11200 MHz

Found 255 files with ≥50% overlap
Overlap percentage range: 66.9% - 100.0%

Top files by overlap percentage:
  spliced_blc00010203040506o7o0111213141516o7o031323334353637_guppi_58328_47198_HIP3909_0036.gpuspec.0000.h5: 7501.5-11251.5 MHz (100.0% overlap)
  spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58417_13701_HIP112870_0013.gpuspec.0000.h5: 7501.5-11251.5 MHz (100.0% overlap)
  spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58513_06113_HIP26228_0058.gpuspec.0000.h5: 7501.5-11251.5 MHz (100.0% overlap)
  spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58223_01045_HIP26686_0066.gpuspec.0000.h5: 7501.5-11251.5 MHz (100.0% overlap)
  spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58511_42568_HIP56242_0034.gpuspec.0000.h5: 7501.5-11251.5 MHz (100.0% overlap)

Found 255 files with ≥50% overlap
Overlap percentage range: 66.9% - 100.0%

Top files by overlap percentage:
  spl

## 4. Sample Random Files

Randomly select 3 files from the filtered dataset for detailed satellite analysis.

In [5]:
# Number of files to sample
n_sample = 3

if len(filtered_df) >= n_sample:
    # Randomly sample files
    sample_df = filtered_df.sample(n=n_sample, random_state=42).copy()
else:
    # If we don't have enough files, take all of them
    sample_df = filtered_df.copy()
    print(f"Warning: Only {len(filtered_df)} files available, using all of them")

print(f"Selected {len(sample_df)} files for satellite analysis:")
print("="*80)

selected_files = []
for idx, row in sample_df.iterrows():
    file_info = {
        'file_path': row['file'],
        'filename': Path(row['file']).name,
        'ra_deg': row['RA'],
        'dec_deg': row['DEC'],
        'tstart_mjd': row['tstart'],
        'freq_start_mhz': row['freq_start_mhz'],
        'freq_end_mhz': row['freq_end_mhz'],
        'overlap_percentage': row['overlap_percentage'],
        'time_completed': row['time_completed_dt']
    }
    selected_files.append(file_info)
    
    print(f"File: {file_info['filename']}")
    print(f"  RA/DEC: {file_info['ra_deg']:.3f}°, {file_info['dec_deg']:.3f}°")
    print(f"  Start time (MJD): {file_info['tstart_mjd']:.6f}")
    print(f"  Frequency: {file_info['freq_start_mhz']:.1f}-{file_info['freq_end_mhz']:.1f} MHz")
    print(f"  Target overlap: {file_info['overlap_percentage']:.1f}%")
    print()

print(f"Total selected files: {len(selected_files)}")

Selected 3 files for satellite analysis:
File: spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58247_35418_HIP88601_0053.gpuspec.0000.h5
  RA/DEC: 18.091°, 2.496°
  Start time (MJD): 58247.409931
  Frequency: 7501.5-11251.5 MHz
  Target overlap: 100.0%

File: spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58511_42568_HIP56242_0034.gpuspec.0000.h5
  RA/DEC: 11.529°, 14.362°
  Start time (MJD): 58511.492685
  Frequency: 7501.5-11251.5 MHz
  Target overlap: 100.0%

File: spliced_blc00010203040506o7o0111213141516o7o021222324252627_guppi_58445_82374_HIP112870_0104.gpuspec.0000.h5
  RA/DEC: 22.857°, 13.971°
  Start time (MJD): 58445.953403
  Frequency: 7797.4-11102.1 MHz
  Target overlap: 83.7%

Total selected files: 3


## 5. Load Satellite Data

Load Starlink satellite orbital data using Skyfield. We'll use current TLE data from CelesTrak.

In [ ]:
# Initialize Skyfield loader
ts = load.timescale()

# Check the date range of our observations first
print("Checking observation date range...")
obs_mjds = [file_info['tstart_mjd'] for file_info in selected_files]
obs_dates = [datetime(1858, 11, 17) + timedelta(days=mjd) for mjd in obs_mjds]
min_date, max_date = min(obs_dates), max(obs_dates)
print(f"Observation date range: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")

# IMPORTANT: Starlink constellation began in 2019
starlink_start_date = datetime(2019, 5, 23)
if min_date < starlink_start_date:
    print(f"\n⚠️  Some observations are from before {starlink_start_date.strftime('%Y-%m-%d')} when Starlink didn't exist!")

def estimate_starlink_count_by_date(target_date):
    """Estimate number of operational Starlink satellites by date"""
    if target_date < starlink_start_date:
        return 0
    
    # Rough timeline of Starlink constellation growth
    launches = [
        (datetime(2019, 5, 24), 60),    # First launch
        (datetime(2019, 11, 11), 120),  # Second launch  
        (datetime(2020, 1, 7), 180),    # Third launch
        (datetime(2020, 6, 1), 500),    # Multiple launches
        (datetime(2021, 1, 1), 1000),   # Rapid expansion
        (datetime(2022, 1, 1), 2000),   # Continued growth
        (datetime(2023, 1, 1), 3500),   # Major expansion
        (datetime(2024, 1, 1), 5000),   # Current scale
        (datetime(2025, 1, 1), 6000),   # Recent numbers
    ]
    
    for date, count in launches:
        if target_date < date:
            return count
    return 6000  # Current estimate

def get_historical_starlink_tles(target_date):
    """
    Fetch Starlink TLE data appropriate for the target date.
    Uses multiple strategies for historical data.
    """
    print(f"  Fetching TLEs for {target_date.strftime('%Y-%m-%d')}...")
    
    # If observation is before Starlink existed, return empty
    if target_date < starlink_start_date:
        print(f"    No Starlink satellites existed on {target_date.strftime('%Y-%m-%d')}")
        return []
    
    expected_count = estimate_starlink_count_by_date(target_date)
    print(f"    Expected ~{expected_count} Starlink satellites for this date")
    
    # Strategy 1: Try current TLE data (best we can do without credentials)
    # In production, you would use Space-Track.org with historical queries
    urls_to_try = [
        'https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle',
    ]
    
    for url in urls_to_try:
        try:
            print(f"    Fetching from: {url}")
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            
            # Parse TLE data
            tle_lines = response.text.strip().split('\n')
            satellites = []
            
            # Group TLE lines into sets of 3 (name, line1, line2)
            for i in range(0, len(tle_lines), 3):
                if i + 2 < len(tle_lines):
                    name = tle_lines[i].strip()
                    line1 = tle_lines[i + 1].strip()
                    line2 = tle_lines[i + 2].strip()
                    
                    # Create satellite object
                    if line1.startswith('1 ') and line2.startswith('2 '):
                        try:
                            from skyfield.sgp4lib import EarthSatellite
                            sat = EarthSatellite(line1, line2, name, ts)
                            
                            # For historical dates, we would ideally filter to only
                            # include satellites that existed at that time
                            # For now, we'll use all current satellites with a warning
                            satellites.append(sat)
                        except Exception as e:
                            continue  # Skip invalid TLEs
            
            if satellites:
                actual_count = len(satellites)
                print(f"    Loaded {actual_count} satellites (current TLE data)")
                
                if target_date < datetime(2024, 1, 1):
                    # For historical dates, subsample to approximate constellation size
                    if actual_count > expected_count and expected_count > 0:
                        # Randomly subsample to approximate historical constellation size
                        import random
                        random.seed(int(target_date.timestamp()))  # Deterministic sampling
                        satellites = random.sample(satellites, min(expected_count, actual_count))
                        print(f"    Subsampled to {len(satellites)} satellites (estimated for {target_date.strftime('%Y-%m-%d')})")
                
                return satellites
                
        except Exception as e:
            print(f"    Failed: {e}")
            continue
    
    print(f"    Could not fetch TLEs for {target_date.strftime('%Y-%m-%d')}")
    return []

# Group observations by date to minimize TLE downloads
print(f"\n📡 Loading Historical TLE Data")
print("=" * 50)
obs_by_date = {}
for file_info in selected_files:
    obs_date = datetime(1858, 11, 17) + timedelta(days=file_info['tstart_mjd'])
    date_key = obs_date.strftime('%Y-%m-%d')
    if date_key not in obs_by_date:
        obs_by_date[date_key] = []
    obs_by_date[date_key].append(file_info)

print(f"Found observations on {len(obs_by_date)} different dates:")
for date_str, files in obs_by_date.items():
    print(f"  {date_str}: {len(files)} observations")

# Load TLEs for each unique date
starlink_by_date = {}
for date_str, files_on_date in obs_by_date.items():
    obs_date = datetime.strptime(date_str, '%Y-%m-%d')
    print(f"\nProcessing {date_str}...")
    
    starlink_sats = get_historical_starlink_tles(obs_date)
    if starlink_sats:
        starlink_by_date[date_str] = {sat.name: sat for sat in starlink_sats}
        print(f"  ✅ Cached {len(starlink_sats)} satellites for {date_str}")
    else:
        starlink_by_date[date_str] = {}
        print(f"  ❌ No satellites available for {date_str}")

# Summary
total_unique_sats = set()
for date_sats in starlink_by_date.values():
    total_unique_sats.update(date_sats.keys())

print(f"\n🎯 TLE Loading Summary")
print("=" * 30)
print(f"Dates processed: {len(obs_by_date)}")
print(f"Dates with satellite data: {len([d for d, sats in starlink_by_date.items() if sats])}")
print(f"Total unique satellites: {len(total_unique_sats)}")

# Show constellation evolution
if len(starlink_by_date) > 1:
    print(f"\nConstellation Evolution:")
    for date_str in sorted(starlink_by_date.keys()):
        sat_count = len(starlink_by_date[date_str])
        obs_count = len(obs_by_date[date_str])
        print(f"  {date_str}: {sat_count:4d} satellites, {obs_count} observations")

if len(total_unique_sats) == 0:
    print("\n⚠️  No satellite data loaded. Creating demonstration satellites...")
    from skyfield.sgp4lib import EarthSatellite
    
    # Create multiple demo satellites for different dates
    demo_satellites = []
    for i in range(3):
        line1 = f"1 4471{i}U 19074A   21200.12345678  .00001234  00000-0  12345-4 0  9999"
        line2 = f"2 4471{i}  53.0538 {123.4567 + i*10:.4f} 0001234  {12.3456 + i*5:.4f} 347.6543 15.05123456123456"
        sat = EarthSatellite(line1, line2, f"STARLINK-DEMO-{i+1}", ts)
        demo_satellites.append(sat)
    
    # Add demo satellites to all dates that need them
    for date_str in starlink_by_date.keys():
        if not starlink_by_date[date_str]:
            starlink_by_date[date_str] = {sat.name: sat for sat in demo_satellites}
    
    print(f"   Added {len(demo_satellites)} demonstration satellites")

In [ ]:
# Filter observations to only include those from when Starlink existed
starlink_start_mjd = 58622  # May 23, 2019 (first Starlink launch)

print("Filtering observations to Starlink era...")
print(f"Starlink first launch: MJD {starlink_start_mjd} (May 23, 2019)")

# Filter selected files to only include post-Starlink observations
original_count = len(selected_files)
selected_files_filtered = [
    file_info for file_info in selected_files 
    if file_info['tstart_mjd'] >= starlink_start_mjd
]

print(f"\nObservation filtering results:")
print(f"  Original files: {original_count}")
print(f"  Files from Starlink era: {len(selected_files_filtered)}")
print(f"  Files filtered out: {original_count - len(selected_files_filtered)}")

if len(selected_files_filtered) == 0:
    print("\n⚠️  No observations from the Starlink era found in sample!")
    print("   Consider selecting different files or expanding the sample size.")
    print("   For demonstration, we'll proceed with all files but note the limitations.")
    selected_files_filtered = selected_files
    
    print("\n📝 Analysis Notes:")
    print("   - Pre-2019 observations: No Starlink satellites existed")
    print("   - 2019-2021 observations: Fewer Starlink satellites")
    print("   - 2022+ observations: Full Starlink constellation")
else:
    print(f"\nProceeding with {len(selected_files_filtered)} files from the Starlink era.")

# Update our working list
selected_files = selected_files_filtered

### ⚠️ Important Note on Historical TLE Data

**Current Limitation:** This notebook uses current TLE (Two-Line Element) data to predict satellite positions for historical observations. This approach has significant limitations:

1. **Temporal Accuracy**: Satellite orbits decay and change over time. Using 2025 TLE data to predict 2020 positions can introduce errors of several degrees.

2. **Constellation Evolution**: The Starlink constellation has grown from 60 satellites in 2019 to thousands today. Historical analysis should account for the actual number of satellites operational at each observation time.

3. **Orbital Maneuvers**: Satellites perform orbit-raising and maintenance maneuvers that aren't captured in current TLEs.

**For Production Analysis, Consider:**

- **Historical TLE Archives**: Use services like Space-Track.org or CelesTrak archives to get TLE data from the observation epoch
- **Constellation Timeline**: Track when each satellite was launched and became operational
- **TLE Epoch Matching**: Use TLE data within days/weeks of the observation time
- **Multiple Constellations**: Include other satellite systems (GPS, communication satellites) that existed throughout the observation period

**This notebook provides a framework that can be extended with proper historical TLE data for more accurate analysis.**

## 6. Calculate Satellite Positions

For each selected observation file, calculate the positions of all Starlink satellites at the observation time.

In [ ]:
from astropy.coordinates import EarthLocation
import astropy.units as u

gbt_loc = EarthLocation.of_site('Green Bank Telescope')

In [ ]:
# Green Bank Observatory coordinates (from the telescope metadata)
# These coordinates are approximate for the GBT
# gbt_lat = 38.4331  # degrees North
# gbt_lon = -79.8397  # degrees West  
# gbt_elevation = 824  # meters
gbt_lat = gbt_loc.lat.to_value(u.deg)
gbt_lon = gbt_loc.lon.to_value(u.deg)
gbt_elevation = gbt_loc.height.to_value(u.m)

# Create observatory location
observatory = Topos(latitude_degrees=gbt_lat, 
                   longitude_degrees=gbt_lon, 
                   elevation_m=gbt_elevation)

print(f"Observatory location: {gbt_lat:.4f}°N, {gbt_lon:.4f}°W, {gbt_elevation}m")

def mjd_to_skyfield_time(mjd):
    """Convert Modified Julian Date to Skyfield Time object"""
    # MJD = JD - 2400000.5
    jd = mjd + 2400000.5
    return ts.tt_jd(jd)


def calculate_angular_separation(ra1_deg, dec1_deg, ra2_deg, dec2_deg):
    """Calculate angular separation between two sky positions in degrees"""
    # Convert to radians
    ra1, dec1 = np.radians(ra1_deg), np.radians(dec1_deg)
    ra2, dec2 = np.radians(ra2_deg), np.radians(dec2_deg)
    
    # Haversine formula for angular separation
    dra = ra2 - ra1
    ddec = dec2 - dec1
    
    a = (np.sin(ddec/2)**2 + 
         np.cos(dec1) * np.cos(dec2) * np.sin(dra/2)**2)
    c = 2 * np.arcsin(np.sqrt(a))
    
    return np.degrees(c)

print("Functions defined for satellite position calculations.")

## 7. Find Nearby Satellites

Calculate angular separations between each satellite and the observation beam center, and identify satellites within 1 degree.

In [ ]:
# Maximum separation to consider (in degrees)
max_separation_deg = 1.0

satellite_crossings = []

print(f"Analyzing satellite positions for {len(selected_files)} observation files...")
print(f"Looking for satellites within {max_separation_deg}° of beam center")
print("="*80)

for i, file_info in enumerate(selected_files):
    print(f"\nFile {i+1}/{len(selected_files)}: {file_info['filename']}")
    print(f"RA/DEC: {file_info['ra_deg']:.3f}°, {file_info['dec_deg']:.3f}°")
    print(f"Observation time (MJD): {file_info['tstart_mjd']:.6f}")
    
    # Get the observation date for TLE lookup
    obs_date = datetime(1858, 11, 17) + timedelta(days=file_info['tstart_mjd'])
    date_key = obs_date.strftime('%Y-%m-%d')
    print(f"Observation date: {date_key}")
    
    # Get satellite data for this observation date
    starlink_dict = starlink_by_date.get(date_key, {})
    if not starlink_dict:
        print(f"  No satellite data available for {date_key}")
        satellite_crossings.append({
            'file_info': file_info,
            'nearby_satellites': [],
            'num_nearby_satellites': 0,
            'min_separation_deg': None,
            'closest_satellite': None
        })
        continue
    
    print(f"  Using {len(starlink_dict)} satellites from {date_key} TLE data")
    
    # Convert observation time to Skyfield format
    obs_time = mjd_to_skyfield_time(file_info['tstart_mjd'])
    
    nearby_satellites = []
    min_separation = float('inf')
    closest_satellite = None
    
    # Check each Starlink satellite for this date
    satellites_checked = 0
    satellites_computed = 0
    
    for sat_name, satellite in starlink_dict.items():
        satellites_checked += 1
        try:
            # Get satellite position at observation time as seen from the observatory
            geometry = satellite.at(obs_time)
            topocentric = geometry - observatory.at(obs_time)
            
            # Get RA/Dec coordinates
            ra, dec, distance = topocentric.radec()
            satellites_computed += 1
            
            # Calculate angular separation from observation beam center
            separation = calculate_angular_separation(
                file_info['ra_deg'], file_info['dec_deg'],
                ra.degrees, dec.degrees
            )
            
            # Check if within threshold
            if separation <= max_separation_deg:
                sat_info = {
                    'satellite_name': sat_name,
                    'separation_deg': separation,
                    'sat_ra_deg': ra.degrees,
                    'sat_dec_deg': dec.degrees,
                    'sat_distance_km': distance.km,
                    'observation_date': date_key
                }
                nearby_satellites.append(sat_info)
                
                # Track closest satellite
                if separation < min_separation:
                    min_separation = separation
                    closest_satellite = sat_name
                    
        except Exception as e:
            # Skip satellites that can't be computed (e.g., due to epoch issues)
            continue
    
    print(f"  Checked {satellites_checked} satellites, computed {satellites_computed} positions")
    
    # Store results for this file
    crossing_info = {
        'file_info': file_info,
        'nearby_satellites': nearby_satellites,
        'num_nearby_satellites': len(nearby_satellites),
        'min_separation_deg': min_separation if min_separation != float('inf') else None,
        'closest_satellite': closest_satellite,
        'observation_date': date_key,
        'satellites_available': len(starlink_dict)
    }
    satellite_crossings.append(crossing_info)
    
    # Print results for this file
    if len(nearby_satellites) > 0:
        print(f"  Found {len(nearby_satellites)} satellites within {max_separation_deg}°:")
        print(f"    Closest: {closest_satellite} at {min_separation:.3f}°")
        
        # Show up to 5 closest satellites
        sorted_sats = sorted(nearby_satellites, key=lambda x: x['separation_deg'])
        for j, sat in enumerate(sorted_sats[:5]):
            print(f"      {sat['satellite_name']}: {sat['separation_deg']:.3f}° separation")
        if len(sorted_sats) > 5:
            print(f"      ... and {len(sorted_sats) - 5} more")
    else:
        print(f"  No satellites found within {max_separation_deg}°")

print(f"\nCompleted satellite analysis for all {len(selected_files)} files.")

# Summary of TLE usage
tle_dates_used = set(crossing['observation_date'] for crossing in satellite_crossings)
print(f"\nTLE Data Summary:")
print(f"  Unique observation dates: {len(tle_dates_used)}")
for date in sorted(tle_dates_used):
    num_sats = len(starlink_by_date.get(date, {}))
    files_on_date = len([c for c in satellite_crossings if c['observation_date'] == date])
    print(f"    {date}: {num_sats} satellites, {files_on_date} observations")

## 8. Analyze Results

Create a summary of satellite crossings and visualize the findings.

In [ ]:
# Create summary statistics
print("SATELLITE CROSSING ANALYSIS SUMMARY")
print("="*50)

total_files = len(satellite_crossings)
files_with_satellites = sum(1 for crossing in satellite_crossings if crossing['num_nearby_satellites'] > 0)
total_satellite_detections = sum(crossing['num_nearby_satellites'] for crossing in satellite_crossings)

print(f"Total files analyzed: {total_files}")
print(f"Files with nearby satellites: {files_with_satellites}")
print(f"Percentage with satellites: {(files_with_satellites/total_files)*100:.1f}%")
print(f"Total satellite detections: {total_satellite_detections}")

if files_with_satellites > 0:
    avg_satellites_per_file = total_satellite_detections / files_with_satellites
    print(f"Average satellites per file (with detections): {avg_satellites_per_file:.1f}")

print("\nDETAILED RESULTS:")
print("-" * 40)

summary_list = []

for i, crossing in enumerate(satellite_crossings):
    file_info = crossing['file_info']
    print(f"\nFile {i+1}: {file_info['filename']}")
    print(f"  Position: RA={file_info['ra_deg']:.3f}°, DEC={file_info['dec_deg']:.3f}°")
    print(f"  Frequency overlap: {file_info['overlap_percentage']:.1f}%")
    print(f"  Nearby satellites: {crossing['num_nearby_satellites']}")
    
    if crossing['closest_satellite']:
        print(f"  Closest satellite: {crossing['closest_satellite']}")
        print(f"  Minimum separation: {crossing['min_separation_deg']:.3f}°")
    
    # Add to summary list
    summary_dict = {
        'filename': file_info['filename'],
        'ra_deg': file_info['ra_deg'],
        'dec_deg': file_info['dec_deg'],
        'tstart_mjd': file_info['tstart_mjd'],
        'freq_overlap_pct': file_info['overlap_percentage'],
        'num_nearby_satellites': crossing['num_nearby_satellites'],
        'closest_satellite': crossing['closest_satellite'],
        'min_separation_deg': crossing['min_separation_deg']
    }
    summary_list.append(summary_dict)

# Convert to DataFrame for easy manipulation
summary_df = pd.DataFrame(summary_list)

print(f"\nSummary DataFrame:")
print(summary_df.to_string(index=False))

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Satellite Crossing Analysis Results', fontsize=16)

# 1. Sky positions of observations
ax1 = axes[0, 0]
colors = ['red' if n > 0 else 'blue' for n in summary_df['num_nearby_satellites']]
scatter = ax1.scatter(summary_df['ra_deg'], summary_df['dec_deg'], 
                     c=colors, s=100, alpha=0.7)
ax1.set_xlabel('Right Ascension (degrees)')
ax1.set_ylabel('Declination (degrees)')
ax1.set_title('Observation Positions\n(Red: with satellites, Blue: no satellites)')
ax1.grid(True, alpha=0.3)

# Add labels for each point
for i, row in summary_df.iterrows():
    ax1.annotate(f'F{i+1}', (row['ra_deg'], row['dec_deg']), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

# 2. Number of nearby satellites per file
ax2 = axes[0, 1]
bars = ax2.bar(range(len(summary_df)), summary_df['num_nearby_satellites'], 
               color=['red' if n > 0 else 'gray' for n in summary_df['num_nearby_satellites']])
ax2.set_xlabel('File Index')
ax2.set_ylabel('Number of Nearby Satellites')
ax2.set_title('Satellites Within 1° of Each Observation')
ax2.set_xticks(range(len(summary_df)))
ax2.set_xticklabels([f'F{i+1}' for i in range(len(summary_df))])

# 3. Minimum separation distances (for files with satellites)
ax3 = axes[1, 0]
files_with_sats = summary_df[summary_df['num_nearby_satellites'] > 0]
if len(files_with_sats) > 0:
    bars = ax3.bar(range(len(files_with_sats)), files_with_sats['min_separation_deg'], 
                   color='orange', alpha=0.7)
    ax3.set_xlabel('File Index (with satellites)')
    ax3.set_ylabel('Minimum Separation (degrees)')
    ax3.set_title('Closest Satellite Distance')
    ax3.set_xticks(range(len(files_with_sats)))
    file_indices = [i+1 for i, row in summary_df.iterrows() if row['num_nearby_satellites'] > 0]
    ax3.set_xticklabels([f'F{i}' for i in file_indices])
    ax3.axhline(y=max_separation_deg, color='red', linestyle='--', 
                label=f'{max_separation_deg}° threshold')
    ax3.legend()
else:
    ax3.text(0.5, 0.5, 'No satellites found\nwithin 1° of any observation', 
             transform=ax3.transAxes, ha='center', va='center', fontsize=12)
    ax3.set_title('Closest Satellite Distance')

# 4. Frequency overlap distribution
ax4 = axes[1, 1]
bins = np.linspace(summary_df['freq_overlap_pct'].min(), 
                   summary_df['freq_overlap_pct'].max(), 10)
ax4.hist(summary_df['freq_overlap_pct'], bins=bins, alpha=0.7, color='green', edgecolor='black')
ax4.set_xlabel('Frequency Overlap with 10.6-11.2 GHz (%)')
ax4.set_ylabel('Number of Files')
ax4.set_title('Target Frequency Range Overlap')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save results to file for future reference
output_file = Path.cwd() / 'satellite_crossing_results.csv'
summary_df.to_csv(output_file, index=False)
print(f"\nResults saved to: {output_file}")

## Conclusions and Next Steps

This notebook demonstrates how to correlate RFI pipeline observations with Starlink satellite positions to identify potential interference sources. 

### Key Findings:
- We analyzed observations with significant overlap in the 10.6-11.2 GHz frequency range
- For each observation, we calculated satellite positions and identified those within 1° of the beam center
- The analysis provides satellite names, angular separations, and detailed position information

### Potential Improvements:
1. **Time Resolution**: Consider the full observation duration, not just the start time
2. **Beam Pattern**: Account for the actual telescope beam shape and sensitivity pattern
3. **Satellite Movement**: Track satellite motion during the observation period
4. **Historical TLEs**: Use TLE data that matches the observation epoch for better accuracy
5. **Other Satellite Constellations**: Extend analysis to include OneWeb, Amazon Kuiper, etc.
6. **Correlation with RFI Detections**: Compare satellite positions with actual RFI hit locations

### Data Products:
- `satellite_crossings`: Complete analysis results for each file
- `summary_df`: DataFrame with key metrics for each observation
- `satellite_crossing_results.csv`: Saved summary for future analysis

This framework can be extended to process larger datasets and provide systematic satellite interference monitoring for radio astronomy observations.